# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print('到達頁面底端')
        break

    print('網頁更新中...')
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 帝寶將掀喬遷潮！南港世界明珠300億元聯貸　今日簽約

▲世界明珠開發案新台幣300億元聯貸案，南榮開發名譽董事長林學圃（右）、董事長張雲鵬（左）共同簽署。（圖／記者吳靜君攝）
記者吳靜君／台北報導
南港開發案「世界明珠」聯貸案由華銀主辦並且統籌其他13家金融機構參貸，華銀與南榮開發、元瑞開發以及智凱開發簽約。南榮開發名譽董事長林學圃今（23）日笑說：「等世界明珠完工之後，仁愛帝寶會掀起一股遷徙潮」。
華銀開發案統籌主辦300億元「世界明珠」建案聯貸案，並且在今日舉行授信簽約，由華南銀董事長張雲鵬、林學圃、董事長張昌平、元瑞開發與智凱開發董事長王忠正及13家金融機構參加。
張雲鵬表示，世界明珠是華南近年辦理最大一件聯貸案，南港為台北的新興發展地區，在5年、10年後，發展可能就跟現在的信義區一樣，林學圃看好世界明珠蓋好之後，可能產生一波「帝寶遷徙潮」，吸引金字塔頂端的客戶入住。
張雲鵬說，世界明珠開發案占地9000坪，是少見的大面積土地，總樓地板面積7萬坪，共8棟建築物，其中有2棟商辦落成後將成為南港首屈一指的頂級A辦，聯強也攜手華邦電以每坪百萬元購買15層，已經引起市場的關注。此外，近期住宅部分也賣出53戶、總計100多億元，華銀認為，該地本身土地價值就很高，就擔保程度來講也相當高。
此建案也聘請華熊營造、中鹿營造共同承攬工程，華熊營造是日本熊谷組在台子公司，過去承攬台北101、新光摩天樓和陶朱隱園；中鹿營造則是日本鹿兒島公司在台子公司，承攬過宏盛帝寶、台北文創以及天母SOGO、華新麗華和台北轉運站等工程。世界明珠是由一手打造頂級豪宅宏盛帝寶的執行長洪國雄操刀規劃，超越帝寶更是期待。
華銀表示，「世界明珠」開發案所需資金，由華南銀行統籌主辦並擔任管理行、台灣銀行共同主辦，參貸行包括台企、彰銀、新光、上海、合庫、安泰及農業金庫等7家銀行，以及國票、兆豐票、萬通票、大中票及合庫票等5家票券公司，短期內即完成300億元募集。
----------------------------------------------------------------------

[1] 外資放假去！台股進入作夢行情　法人看好後市3題材

▲蝦皮購物整理全站年度消費數據，分享2019年電商消費趨勢的變化。（圖／蝦皮購物提供）
記者洪菱鞠／台北報導
網購熱潮持續升溫，今年雙11、雙12，各家電商又再刷出新一波數據紀錄，除了反應消費者比去年更愛買，網購業者更發現到消費輪廓有大洗牌現象，網購已不再是「年輕」、「女性」的專利，隨著「熟齡」、「男性」挾帶高含金量大量湧入，成為電商市場新金礦；另外也發現到一項有趣現象，「情趣用品」不僅成為熱搜關鍵字之一，搜尋次數更達到近100萬次。
蝦皮購物透過訂單數量、商品數量、關鍵字等角度，整理全站年度消費數據，其中觀察到，2019年不同性別、年齡的消費者皆有成長，但男性消費者不僅在「數量」的年度成長率大幅優於女性、「訂單總數」的年度成長率更是女性的1.6倍。「女性、年輕」族群雖持續成長、貢獻消費，卻不再一枝獨秀，「男性、熟齡」族群反為後起之秀，而進一步分析男性消費者組成，其中又以50歲～64歲區間成長幅度最高，男性熟齡消費者在2019年的數量亦是去年的1.6倍。
綜觀全年度消費，「居家生活」、「美妝保健」向來是熱銷常勝軍，但蝦皮購物統計2019年各品類訂單數量，相較去年成長最多的前5名為「家電影音」、「遊戲」、「戶外與運動用品」、「汽機車百貨」、「手機平板與周邊」，5大品類的年度訂單量較去年創下飛躍式成長，呈現出令人意外的結果。
由於越來越多消費者選擇從電商網站購買家電，家庭、隨身影音設備成為近年大勢，訂單量飆出年度最高成長；此外，去年引起旋風的任天堂Switch人氣絲毫未減，Switch主機熱賣帶動相關遊戲保持極高的發行頻率，「有新品就有新訂單」的策略，刺激「遊戲」類商品一口氣衝上年度訂單成長第2名；「戶外與運動」商品的熱銷，也具體反映出「健身、運動」風潮席捲全台，促使更多消費者從電商平台購入相關商品。
在搜尋方面，蝦皮購物統整2019年「關鍵字搜尋排行榜Top10」，強勢登上冠軍的關鍵字是「耳機」，「手機殼」、「airpods」則緊追在後，前3名皆與手機相關，另也意外發現到「情趣用品」相關關鍵字名列其中，搜尋次數更達到近100萬次，推測原因與網路購物所具備的隱密性優勢有關，方便消費者能更自在地購入保險套、各種情趣商品，不僅能避免店面購買的尷尬，電商平台在商品種類的多元性、多方比價的便利性皆優於實體通路，這也成為情趣相關商品在電商市場熱銷的重要原因。
▲Dyson吸塵器年

▲保險理賠透過新科技的應用，大幅縮短保戶理賠程序與時間。（示意圖／pixabay）
記者紀佳妘／台北報導
保險理賠近年來搶攻保險科技應用，中信金以區塊鏈的方式，傳遞相關理賠所需的單據，加速判別理賠金額的流程，大幅縮短台灣人壽保戶理賠時間，而富邦人壽則與台中榮總合作，雙方透過資料對接系統，加速理賠審核作業，縮減保戶拿到理賠金的時間。
中信金2016年成立區塊鏈實驗室，成為全台首家加入國際金融區塊鏈聯盟R3的金融業者，近3年來推動Shaman區塊鏈付款確認服務、貿易融資信用狀區塊鏈以及eClaim理賠區塊鏈等3大服務，其中在健康醫療部分，著重在區塊鏈如何串起金融與醫療合作項目，區塊鏈技術能保護用戶的資料授權軌跡與正確性，促進醫療資訊自主權，更落實參與式與人群醫學，具學術研究、醫藥研發價值。
中國信託金控技術長賈景光表示，台灣人壽與高雄榮民總醫院合作的「eClaim理賠區塊鏈」平台即為應用實例，也是全台首創醫療全險種數位理賠服務，基於區塊鏈強化跨業資訊交換的信任度，讓保戶獲得更高效率的理賠服務。
賈景光進一步指出，過去保戶出院要申請醫療理賠時，必須向醫院申請各式書面單據，再向保險公司申請理賠，不過7月1日上線的「eClaim理賠區塊鏈」，只要保戶線上授權後，透過區塊鏈往返傳輸授權文件，台壽即可向醫院拿到單據進行理賠，大幅縮短保戶理賠程序與時間，未來也將持續擴大到非台壽的保戶。
富邦人壽不僅串聯全台醫學網絡推出「安心住院理賠免煩惱專案」，提供住院抵繳服務，更攜手台中榮民總醫院建置資料對接系統升級為「安心住院理賠免煩惱專案2.0」，理賠人員可直接透由雙方打造的資料對接系統，接收保戶住院天數、就診單據等相關資料，並自動試算理賠金額，於保戶出院時扣抵醫療費用，透過保險科技加速理賠審核作業，讓保戶出院後，免於舟車勞頓申請診斷證明等流程，也能安心靜養。
富邦人壽總經理陳俊伴表示，「關懷人」永遠是保險業的初心，未來將秉持此精神，多元應用數位科技掌握服務商機。另隨著數位時代的來臨，透過「虛實整合」將保險科技落實於各大理賠服務之中，像是智能化的「理賠easy pay」，線上上傳單據即可申請理賠，將理賠核發作業時間從3天縮減至0.5天，還能即時追蹤理賠進度。
此外，富邦人壽也提供「視訊理賠服務」，對於身體狀況不便或居住在偏鄉地區的保戶，主動於約定給付日前通知保戶，保戶於給付日使用行動裝置與健

▲外界關注電價調整情況，經濟部長沈榮津今（23）日表示，將尊重審議委員會的審議機制，強調明年電價不會漲五成。（圖／記者湯興漢攝）
記者林淑慧／台北報導
經濟部將於明年四月召開電價費率審議委員會，外界關注電價調整狀況，經濟部長沈榮津今（23）日澄清表示，電價調整尊重審議委員會的審議機制，強調「不是政治人物說了算」，也將考慮民生需求，過年前只會調降不調漲，外界普遍解讀明年電價可望「連四凍」。
經濟部每年召開兩次電價費率審議會，在4月及10月公告是否調整電價，並明定電價調幅最高3%。之前兩次（去年10月、今年4月），國內電價已連續二次凍漲，目前每度平均2.6253元，造成台電上半年大虧，是唯一虧損的國營事業。
經濟部今天與媒體舉行年終餐敘，面對近期電價調整的說法，稱明年電費比現在多五成，經濟部長沈榮津特別澄清，這是造謠，也不可能發生，經濟部謹記過去油電雙漲，對民生物價造成極大衝擊，因此設有電價穩定機制，每次調幅以3%為上限。
沈榮津表示，電價調整將尊重審議委員們的專業意見，但按照過往經驗，審議委員會衡量國際燃料價格、台電發電成本及利潤等面向，而電價費率審議會的委員有來自學界、業界及機關代表，會依據客觀事實和數據資料討論。
經濟部表示，過去油電雙漲嚴重衝擊物價，2014年時，電價一度漲到3.07元，影響民生物價，因此政府設有電價穩定機制，電價審議每年檢討兩次，調幅均不得超過3%，所謂明年多50%的情況不可能發生。
另外，經濟部強調，依照電價穩定機制，住宅用電330度以下（大約七成民生用戶）電費不漲，小商家1,500度以下不漲，可以同時照顧民生、穩定物價及節約用電。
----------------------------------------------------------------------

[29] 哈日族買到便宜日圓有撇步　銀行傳授3招鎖住最佳匯率

▲日本是台灣人最愛的旅遊目的地，日圓匯率的走勢都關係到民眾的荷包。（圖／記者蔡玟君攝）
記者紀佳妘／台北報導
在中美貿易談判取得第一階段協議一下，新台幣近期走升、日圓匯價持續走跌，哈日族期待已久的0.27字頭終於回來了，對於想赴日本旅遊的民眾不妨掌握3個換匯技巧，並搭配刷卡賺回饋，即可省下一筆可觀的費用。
匯率隨時隨地都在變動，目前最常見的換匯管道，包括銀行臨櫃、外幣ATM、線上結匯、從外幣帳戶提款共4種方式

▲財政部發佈的「共同申報及盡職審查作業辦法」、「租稅協定稅務用途資訊交換作業辦法」等等，政府近年在跟進金融合規現代化的實質努力，亦受到APG等國際組織的認可。（圖／記者陳韋帆攝）
●李牮斯／避險基金經理人，新世界智庫籌辦人
亞太金融中心的口號，在台已行之多年，尤其每逢選舉之際則更加被宣揚。但除了台灣保險業在國際債券市場成為了舉足輕重的買家之外，實際上有利於台灣永續發展的金融成果寥寥無幾。這背後的原因諸多，包括台灣的外交困境、不利於國際資產管理產業發展的稅務系統與法制、語言隔閡、以及缺乏專業人才等等，皆是需要排除的阻力。但這些阻礙在近期國際宏觀環境的劇變之下，相形不足掛齒，重點在於台灣是否能凝聚國家上下的共識，步步為營，在外交的絕地上重生新金融產業。
▲「CRS共同申報準則」，會員國互相分享透過銀行金融系統所收集的國內外客戶資訊，圍堵避稅渠道以補國庫。（圖／視覺中國CFP）
台灣長年不懈貢獻於國際社會，但試著加入各種國際組織時常受排擠，CRS也不例外。在反恐反洗錢反避稅的精神下，自2017年起台灣積極更新了諸多呼應CRS要求的相關法案，如立法院「稅捐稽徵法修正案」與財政部發佈的「共同申報及盡職審查作業辦法」、「租稅協定稅務用途資訊交換作業辦法」等等，政府近年在跟進金融合規現代化的實質努力，亦受到APG等國際組織的認可。台灣盡國際公民之職責，確切執行反恐反洗錢反避稅的資訊收集之實，然而我們卻不被OECD提名於CRS名單，也無法以國家名義簽署金融資訊互換協定，因此台灣在提供個人金融帳戶資訊予其他國家的執行上，有很大的不便。
從2017修法至今，僅日本與澳洲兩國參與台灣版本的CRS資訊互換，而兩國以民間組織為代表簽署的合約，仍可能造成執行上法律效力與互換手續的挑戰。然而，上述未竟之功也可以不必被視為是我們國際參與的挫敗。因為雖然台灣在CRS架構內於名與實皆陷於模糊狀態，這種狀態可能是許多亟欲發展金融產業的國家夢寐以求的。國際加諸於台灣的窘境，意外地成就了我們保護個人資訊隱私的能力，而隱私是私人銀行與資產管理業最珍貴的競爭壁壘，也是在這CRS世代最稀有的資源，若政府與民間能夠齊心經營，則台灣有潛力能吸引大量國際資本，以新瑞士此一新姿態登上國際舞台。
▲若政府與民間能夠齊心經營，則台灣有潛力能吸引大量國際資本，以新瑞士此一新姿態登上國際舞台。（圖／記者周宸亘攝）
在這法規、政治

...

▲不少銀行趁年底推出美元優利定存專案，其中利率最高喊到5.5%，吸引不少投資人目光。（示意圖／路透社）
記者紀佳妘／台北報導
美元定存向來是台灣人偏愛的外幣投資選項，國銀、外銀近期紛紛推出高利美元定存搶市，包括台新銀、玉山銀、花旗銀等，甚至有銀行短天期年息喊到5.5%，吸引投資人目光。
花旗銀近期推出「紐幣／澳幣／加幣／美元／英鎊定存限時高利」活動，其中美元定存部分，以新資金轉換為美元後辦理定存，且單筆存滿3萬美元，單周（7天期）存款年息5.5%、雙周（14天期）存款年息5%；至於既有資金，單周存款年息4%、雙周存款年息3.5%。
此外，花旗銀行表示，客戶若將資金轉換為人民幣、港幣、歐元，轉換金額達活動金額，當月即可享美金新資金的高利定存，單週定存最高年息6.08%。
玉山銀行12月限定外幣優惠方案，其中美元部分，單筆需存滿1,000美元，透過網銀換匯，享有匯價減碼3.5分優惠，6個月期存款年息1.8%、12個月年息1.9%；此外，玉山推聖誕特別企劃，單筆購買新台幣1萬元（含）起，立即享有匯價減碼4分優惠，不過有限時限量，有需求的民眾不妨可把握機會。
台新銀行祭出「美利升活外幣活存專案」，提供美元活存利率優惠5個月，年息為2.5%。以美元為例，專案年息是一般美元活存牌告的11倍，即使與5個月期美元定存牌告比較，也整整高出2倍，而且除了利率外，民眾若以外幣現鈔存入，或外幣匯入匯款承作專案可減免手續費；如以新台幣資金換匯承作專案者，提供即期牌價美元匯差減降3.5分、人民幣匯差減降0.5分等優惠，承作門檻低3,000美元。
台新銀行指出，以10萬美元新資金為例，年息2.5%在5個月後利息收入將較一般美元活存多出約950美元，換算成新台幣有近3萬元，相當於多了一趟出國旅遊基金，在過程中若臨時有資金需求，隨時可提領款項，完全沒有定存中途解約利息打折的問題，剩餘的本金仍可繼續享有優惠利率。
不過，市場上美元優利定存方案百百種，銀行主管提醒，優利定存通常會有最低承作金額、存款門檻或上限等條件，以存款門檻來說，從1美元到上萬美元都有，因此未必吃得到，投資人應評估自身理財需求與匯率風險，避免被高利沖昏頭。
----------------------------------------------------------------------

[59] 三顧攜手義大醫院《特管法》

▲展望明年金融市場，外銀提醒，投資人想追求穩健的投資組合，須留意3大關鍵問題。（示意圖／pixabay）
記者紀佳妘／台北報導
2019年即將進入尾聲，金融市場繳出漂亮的成績單，對於明（2020）年之後的投資環境仍充滿不確定性，瑞銀預期在寬鬆的貨幣政策下，將在2020年繼續抑制經濟衰退的風險，因此投資人想要追求穩健的投資組合，需要關注3大關鍵問題。
瑞銀財富管理亞太區首席投資總監陳敏蘭（Minlan Tan）表示，面對美國大選、中美貿易戰等不確定因素，預期明年全球經濟增長將低於趨勢水準，退守現金並非良策，投資人反而應該堅持投資，並透過放眼長遠、在地區與資產類別間分散配置和精挑細選來提高投資組合的韌性。
不過，陳敏蘭提醒明年想要建構穩健的投資組合，必須要關注3大關鍵問題。首先是一般國家選舉不會對全球投資人造成太大影響，但美國總統選舉很可能是個例外，隨著大選臨近，預期美國股市的波動可能加劇，尤其是科技、能源、金融和醫療保健行業等主要板塊可能因為監管趨嚴而出現大幅波動，因此投資人不應該押注於美國總統選舉的結果來進行布局。
再者是如何在保護主義盛行的世界中投資，陳敏蘭指出，中美貿易取得第一階段協議，這將為全球股市帶來溫和支撐，但長期立場相當明確，因此即便在美國總統大選之後，也難見到雙方做出重大讓步的跡象。
此外，隨著時間的推移，全球供應鏈和技術標準很可能日益分裂，而供應鏈調整有助於提振大陸以外地區的基建需求，因此新興市場基礎設施可能帶來機會。2020年，東南亞的基建支出料將大幅上升，尤其是在菲律賓、馬來西亞和泰國，同時隨著經濟脫鉤壓力逐漸上升，大陸將為實現自主科技創新而擴大科技投入，相關企業將繼續受惠。
第三是如何在週期後段、收益率走低的環境下投資，陳敏蘭認為，全球經濟即將實現連續10年高於3%的增長，債券收益率接近多年來低點。過去10年中幾乎所有金融資產都取得了強勁的回報，但伴隨經濟週期結束的失衡現象幾乎未見蹤影，比如經濟過熱、消費者槓桿過高或資產泡沫過度等，儘管不認為2020年會出現衰退（概率僅為10%-15%），最有可能的結果是增長放緩或持穩。
在週期後段的低收益率環境下，瑞銀認為股票投資應以品質和股息為重；在亞洲，股息收益率高於其歷史均值，由於派息率依然較低，股息還有增長空間，因此偏好股息率高於政府債券收益率市場中的高股息股票，特別是新加坡、香港、泰國。對於亞洲信用

▲ 在常務理事王令麟(中)見證下，任滿兩屆6年圓滿卸任的台北市商業會理事長王應傑(左)將印信交接給新任理事長吳發添(右)。（圖／台北市商業會提供）
記者余弦妙／台北報導
台北市商業會今（20）日舉辦第18屆第一次會員代表大會，順利改選出33位理事與11位監事，由帆布公會理事長、世運武術護具董事長吳發添，當選新任理事長，台北市家具公會榮譽理事長許俊榮當選監事會召集人，而商業會常務理事王令麟也獲得高票續任。
吳發添是第一批前往中國大陸發展的代表性台商，30年前跟隨其叔父吳盛順（深圳台商協會創會會長）紮根深圳，事業跟隨中國大陸經濟起飛而同步茁壯，長期以來擔任台北市商業會兩岸經貿事務委員會主任委員，致力於推動兩岸經貿交流，同時擔任仲裁協會常務理事與無店面零售商業同業公會常務監事等職，活躍於各工商社團。
吳發添當選後，宣布提名三位常務理事長出任副理事長，分別是陳春銅（良茂機構董事長，台北市建築開發公會榮譽理事長）、涂國安（元茂國際企業董事長，台北市事務器械公會榮譽理事長）以及章金元（元璋玻璃副董事長，台北市玻璃公會榮譽理事長），獲得全體理事一致通過。
▼台北市商業會第十八屆理監事會五位女性成員合影。（圖／台北市商業會提供）
此次換屆改選，總共有五位女性公會理事長當選理監事，是歷來人數最多的一屆，女性理事比例大幅提升，從原本的3位擴大為5位，且理事的類別也趨向更多樣化，這次有旅館公會理事、機場免稅百貨公會理事等加入這次競選。
新任的常務理事孫因（天風設計設計總監，台北市室內設計裝修公會理事長），是第一位女性常務理事；古素琴（彩盟免稅店集團董事長，台北市百貨公會理事長），與邱樂芬（香城飯店集團總監，台北市旅館公會常務監事）則是首次當選的女性生力軍。
理事長吳發添說，這次的選舉過程，除了明顯提升女性比例外，台北商業會也希望可以「進化」，因此這次也特別把較為資深的理事們，改為「榮譽理事」，將與各商業同業公會緊密合作，引進各公會新血理事長，擴大服務團隊陣容，讓六大委員會發揮最大活力，共同爭取商業界合理健康發展空間。
吳發添打趣地說，「這次的選舉多了許多新氣象、氣氛也柔和許多」，意指多了一些女性理事們，也增添一些年輕理事們，讓整個台北市商業會變得更不一樣。吳發添也說，未來也會持續朝這個目標努力，將與各商業同業公會緊密合作，共同爭取商業界合理健康發展空間。
▲台北市商業會第十八屆新任理事長

在股票市場中，除了基本面外，籌碼面也不可或缺。在下一次財報公布前的空窗期中，籌碼的流動將成為主導個股漲跌的致勝關鍵。
文／吳旻蓁
二○二○年在即，距離傳統農曆春節僅剩一個多月，此時不只是投資人幫自己賺紅包的好時機，也是各家投信競爭績效的時刻。這波熱錢潮來勢洶洶，盤面上的強勢標的，除了權值股如台積電、鴻海外，統計最近一個半月以來，投信主要買超的類股來看，從半導體、面板、記憶體、ＰＣＢ、被動元件等產業也都是重點掃貨的對象。
面板雙虎ＰＢ超低
最令人驚訝的是壞了很久的面板廠終於傳來好消息。過去中國為了搶奪市占率，除了仰賴政府的補貼之外，同時也以量價優勢打擊台灣、韓國面板廠，使得ＬＣＤ面板供給大增，造成供過於求，加上今年因貿易戰等區域衝突，導致全球景氣疲軟、消費保守，使得面板報價持續走跌，甚至跌破現金成本。
如今，隨著韓國積極調整產能與關廠、中國面板大廠京東方宣布減產，庫存逐漸降低，加上年底消費旺季需求推動，使面板價格終於在十一月止跌。此外，明年更有東京奧運等大型運動賽事可望帶來一波電視換機潮，需求增加讓面板價格有望於明年第一季反彈；近期也傳出面板廠要調漲價格至現今成本之上，試圖扭轉持續虧損的窘況，利多消息激勵市場信心，推升面板廠友達、群創、彩晶股價群起向上，其中又以面板雙虎友達與群創最受投信喜愛，雙雙買超一萬五千張以上，友達股價更重返十元票面之上，投信也樂觀指出，「面板股最大優勢其實就是基期低，股價淨值比都在○．五倍以下，更何況景氣循環股本來就是買在高本益比、賣在低本益比。」
整體來說，面板供給端的減產或停產，頂多算是給國內面板廠喘口氣的機會，而非解決根本問題，若終端沒有強勁的需求，產能過剩問題只會再次浮現。展望未來，需要新技術的應用以刺激新的市場需求，包括OLED、Mini LED、Micro LED等技術都可望於明、後年逐漸成熟，屆時面板景氣將明顯回溫，早已進場布局高階領域的友達、群創也有望能分一杯羹。
同樣也熬過產業景氣谷底的族群還有記憶體，因美中貿易站因素，自去年下半年開始，客戶減少拉貨，使記憶體庫存水位升高，導致報價急速下跌。
記憶體低庫存迎來春燕
不過隨著負面影響降低，加上明年全球伺服器需求回溫、５Ｇ手機放亮量，以及ＡＩ、物聯網、車用等新型態應用遍地開花，各大投顧包括兆豐、富邦、第一金、玉山等都紛紛押寶，看好記憶體明年成長動能。統一投顧董事長黎方國表示，「低基

美中貿易戰爭，台積電扮演著關鍵角色，尤其5G時代來臨，帶動AI、高速運算（HPC）等應用崛起，當中核心晶片都須高度仰賴台積電的七、五奈米等先進製程，促使台積電訂單滿載，外資也頻頻調升目標價。此外，台積電持續擴大資本支出以及投入先進封裝製程，引發外溢效應，如半導體設備與耗材廠商，以及旗下子公司創意、世界先進與精材等，都牛氣沖天！
文／馮欣仁
十二月十七日，台股護國神山台積電股價再度攻上三四五元，歷史新高價，市值逼近九兆元，居全球第十六大市值企業；同時也推升加權指數成功飛越一萬二千點大關。由於外資看好台積電在七奈米與五奈米訂單與量產速度加快，麥格理證券更出具最新研究報告，將台積電目標價上修至三八五元，居業界最高。除了麥格理之外，瑞信也再度調升其目標價至三七五元；目前花旗、麥格理、瑞士信貸、里昂、高盛、摩根大通和美銀等七大券商，不約而同地將台積電目標價落在三五○～四○○元之間。
七、五奈米領先對手  外資喊讚
台積電的強勢表態，也帶動周邊相關半導體設備、ＩＣ設計、封測等廠商可望雨露均霑。由於台積電在先進製程持續擴大資本支出，激勵應用材料（AMAT）、ASML Holding（ASML）、Teradyne（TER）等生產ＥＵＶ（極紫外光）機台、測試設備等廠商股價走強，連帶國內相關供應鏈如京鼎、漢唐、帆宣、弘塑、家登、日揚、光罩等也因而受惠。其次，則是相關耗材廠商，如再生晶圓中砂與昇陽半導體、特用化學三福化、機台清洗世禾等，也將搭上台積電成長順風車。
外資之所以如此看好台積電未來發展，無非是看中其在先進製程的推進，遙遙領先競爭對手三星與英特爾，更讓蘋果（AAPL）、華為海思、高通（QCOM）、AMD、聯發科等大廠，高度仰賴台積電。過去台積電的製程會依照摩爾定律每二年推進一個世代，不過，未來並非如此。台積電董事長劉德音日前表示，未來不只在拚製程微縮，而是以邏輯密度或運算能力作為指標。台積電預估其五奈米製程晶片邏輯密度是七奈米的一．八倍；在同速度下，功耗降低三○％、在同功耗下，速度提升十五％。目前台積電五奈米製程將於明年上半年正式量產，每月產能由初期五萬片，逐步增加至七萬片，未來有機會上看八萬片產能；屆時對於台積電的整體毛利率與獲利表現具有重大助益。
八吋晶圓吃緊  世界長線營運看俏
台積電這艘超級無敵艦隊，旗下則有三家重要子公司，分別為持股二八．三二％八吋晶圓代工世界、持股四○